In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob
import sys
import time
import numexpr
import bottleneck

In [2]:
def match_key(data) :
    season = data["info"]["season"]
    match_number = None
    if "match_number" in data["info"]["event"] :
        match_number = str(data["info"]["event"]["match_number"])
    elif "stage" in data["info"]["event"] :
        match_number = data["info"]["event"]["stage"]
    return season, match_number

In [3]:
def match_data() :
    match_data_array = []
    match_attrs = ["season", "match_number", "city", "start_date", "winner", "batting_first", "chasing", "eliminator"]
    for filename in glob.glob(os.path.join("ipl_json", '*.json')):
        data = json.load(open(filename))
        city = None
        start_date = None
        winner = None
        eliminator = None
        season, match_number = match_key(data)
        if "city" in data["info"] :
            city = data["info"]["city"]
        if "dates" in data["info"] :
            if len(data["info"]["dates"]) > 0 :
                start_date = data["info"]["dates"][0]
        if "winner" in data["info"]["outcome"] :
            winner = data["info"]["outcome"]["winner"]
        elif "result" in data["info"]["outcome"] :
            winner = data["info"]["outcome"]["result"]
        if "eliminator" in data["info"]["outcome"] :
            eliminator = data["info"]["outcome"]["eliminator"]

        match_data_list = [season, match_number, city,
                   start_date, winner, data["info"]["teams"][0],
                   data["info"]["teams"][1], eliminator]
        match_data_array.append(match_data_list)
    match_df = pd.DataFrame(match_data_array, columns=match_attrs)
    return match_df
match_df = match_data()

In [4]:
def player_data() :
    player_df = pd.read_csv("people.csv")
    player_match_array = []
    full_player_match_attrs = ["name", "player_id", "season", "match_number", "team", "runs_scored", "fours", "sixes", 
                          "out", "balls_faced", "position", "wickets", "runs_conceded", "balls_delivered",
                          "fours_conceded", "sixes_conceded", "wides", "no_balls"]
    player_match_attrs = ["name", "player_id", "season", "match_number", "team"]
    for filename in glob.glob(os.path.join("ipl_json", '*.json')):
        data = json.load(open(filename))
        season, match_number = match_key(data)
        for player in data["info"]["registry"]["people"] :
            team = None
            for team_name in data["info"]["players"] :
                if player in data["info"]["players"][team_name] :
                    team = team_name
            if not team == None : # is an official
                player_match = [player, data["info"]["registry"]["people"][player], season, match_number,
                            team]
                player_match_array.append(player_match)
    player_match_df = pd.DataFrame(player_match_array, columns=player_match_attrs)
    return player_df, player_match_df
player_df, pm_df = player_data()

FileNotFoundError: [Errno 2] No such file or directory: 'people.csv'

In [ ]:
# leaving out the "legal number" field from the delivery array because it's probably unnecessary
# and not worth the trouble
# This function is not vectorized because it doesn't take forever; later functions are vectorized by
# necessity
def delivery_data() :
    delivery_array = []
    delivery_features = ["season", "match_number", "team_batting", "over", "number", "batter",
                         "bowler", "non_striker", "extras", "runs", "total_runs", "wickets", "match_id"]
    wicket_array = []
    wicket_features = ["season", "match_number", "team_batting", "over", "number", "player_out", "type"]
    extra_array = []
    extra_features = ["season", "match_number", "team_batting", "over", "number", "byes", "legbyes", "noballs", "penalty", "wides"]
    fielder_wicket_array = []
    fielder_wicket_features = ["season", "match_number", "team_batting", "over", "number", "id"] # not name, which isn't necessarily unique
    for filename in glob.glob(os.path.join("ipl_json", '*.json')):
        data = json.load(open(filename))
        season, match_number = match_key(data)
        registry = data["info"]["registry"]["people"]
        for innings_data in data["innings"] :
            team_batting = innings_data["team"]
            for over_data in innings_data["overs"] :
                over = over_data["over"]
                for number, delivery_data in enumerate(over_data["deliveries"]) :
                    batter_name = delivery_data["batter"]
                    batter_id = registry[batter_name]
                    bowler_name = delivery_data["bowler"]
                    bowler_id = registry[bowler_name]
                    non_striker_name = delivery_data["non_striker"]
                    non_striker_id = registry[non_striker_name]
                    runs = delivery_data["runs"]["batter"]
                    total_runs = delivery_data["runs"]["total"]
                    wickets = 0
                    extras = delivery_data["runs"]["extras"]

                    if "wickets" in delivery_data :
                        wickets = len(delivery_data["wickets"])
                        for wicket_data in delivery_data["wickets"] :
                            wicket_type = wicket_data["kind"]
                            player_out_name = wicket_data["player_out"]
                            player_out_id = registry[player_out_name]
                            wicket_list = [season, match_number, team_batting, over, number, player_out_id, wicket_type]
                            wicket_array.append(wicket_list)
                            if "fielders" in wicket_data:
                                for fielder_data in wicket_data["fielders"] :
                                    fielder_name = fielder_data["name"]
                                    fielder_id = registry[fielder_name]
                                    fielder_wicket_list = [season, match_number, team_batting, over, number, fielder_id]
                                    fielder_wicket_array.append(fielder_wicket_list)
                    
                    if "extras" in delivery_data :
                        extras_data = delivery_data["extras"]
                        byes, legbyes, noballs, penalty, wides = 0, 0, 0, 0, 0
                        if "byes" in extras_data :
                            byes = extras_data["byes"]
                        if "legbyes" in extras_data :
                            legbyes = extras_data["legbyes"]
                        if "noballs" in extras_data :
                            noballs = extras_data["noballs"]
                        if "penalty" in extras_data :
                            penalty = extras_data["penalty"]
                        if "wides" in extras_data :
                            wides = extras_data["wides"]
                        extra_list = [season, match_number, team_batting, over, number, byes, 
                                       legbyes, noballs, penalty, wides]
                        extra_array.append(extra_list)
                    # In this databases, overs + deliveries are zero indexed
                    delivery_list = [season, match_number, team_batting, over, number, batter_id, 
                                     bowler_id, non_striker_id, extras, runs, total_runs, wickets, str(season) + " " + str(match_number)]
                    delivery_array.append(delivery_list)
    delivery_df = pd.DataFrame(delivery_array, columns=delivery_features)
    wicket_df = pd.DataFrame(wicket_array, columns=wicket_features)
    extra_df = pd.DataFrame(extra_array, columns=extra_features)
    fielder_wicket_df = pd.DataFrame(fielder_wicket_array, columns=fielder_wicket_features)
    return delivery_df, wicket_df, extra_df, fielder_wicket_df
delivery_df, wicket_df, extra_df, fielder_wicket_df = delivery_data()

In [ ]:
def player_scorecards(player_match_df) :    
    # Runs Scored
    runs_scored = delivery_df[["season", "match_number", "batter", "runs"]].groupby(by=["season", "match_number", "batter"], sort=False, as_index=False).sum()
    player_match_df = player_match_df.merge(runs_scored, how="left", left_on=["season", "match_number", "player_id"], right_on=["season", "match_number", "batter"])
    player_match_df = player_match_df.rename(columns={"runs":"runs_scored"})

    # Runs Conceded
    full_extra_df = delivery_df.merge(extra_df, on=["season", "match_number", "team_batting", "over", "number"])
    runs_conceded = delivery_df[["season", "match_number", "bowler", "runs"]].groupby(by=["season", "match_number", "bowler"], sort=False, as_index=False).sum()
    all_extras_conceded = full_extra_df[["season", "match_number", "bowler", "byes", "legbyes", "noballs", "penalty", "wides"]].groupby(by=["season", "match_number", "bowler"], sort=False, as_index=False).sum()
    all_wides_noballs_conceded = all_extras_conceded[["season", "match_number", "bowler", "wides", "noballs"]]
    all_runs_conceded = runs_conceded.merge(all_wides_noballs_conceded, on=["season", "match_number", "bowler"], how="left")
    all_runs_conceded["runs_conceded"] = all_runs_conceded[["runs", "wides", "noballs"]].sum(axis=1)# sum on the hor axis
    player_match_df = player_match_df.merge(all_runs_conceded[["season", "match_number", "bowler", "runs_conceded", "wides", "noballs"]], 
                                            how="left", left_on=["season", "match_number", "player_id"], 
                                            right_on=["season", "match_number", "bowler"])
    player_match_df = player_match_df.drop(["bowler", "batter"], axis=1)

    all_fours = delivery_df.loc[np.where(delivery_df["runs"] == 4)] # yes, I know that they can be run
    all_sixes = delivery_df.loc[np.where(delivery_df["runs"] == 6)]

    # Fours/Sixes Scored/Conceded
    fours_scored = all_fours[["season", "match_number", "batter", "runs"]].groupby(by=["season", "match_number", "batter"], sort=False, as_index=False).count()
    fours_scored = fours_scored.rename(columns={"runs":"fours_scored"})
    sixes_scored = all_sixes[["season", "match_number", "batter", "runs"]].groupby(by=["season", "match_number", "batter"], sort=False, as_index=False).count()
    sixes_scored = sixes_scored.rename(columns={"runs":"sixes_scored"})
    fours_conceded = all_fours[["season", "match_number", "bowler", "runs"]].groupby(by=["season", "match_number", "bowler"], sort=False, as_index=False).count()
    fours_conceded = fours_conceded.rename(columns={"runs":"fours_conceded"})
    sixes_conceded = all_sixes[["season", "match_number", "bowler", "runs"]].groupby(by=["season", "match_number", "bowler"], sort=False, as_index=False).count()
    sixes_conceded = sixes_conceded.rename(columns={"runs":"sixes_conceded"})

    player_match_df = player_match_df.merge(fours_scored, 
                                            how="left", left_on=["season", "match_number", "player_id"], 
                                            right_on=["season", "match_number", "batter"])
    player_match_df = player_match_df.merge(fours_conceded, 
                                            how="left", left_on=["season", "match_number", "player_id"], 
                                            right_on=["season", "match_number", "bowler"])
    player_match_df = player_match_df.drop(["bowler", "batter"], axis=1)

    player_match_df = player_match_df.merge(sixes_scored, 
                                            how="left", left_on=["season", "match_number", "player_id"], 
                                            right_on=["season", "match_number", "batter"])
    player_match_df = player_match_df.merge(sixes_conceded, 
                                            how="left", left_on=["season", "match_number", "player_id"], 
                                            right_on=["season", "match_number", "bowler"])
    player_match_df = player_match_df.drop(["bowler", "batter"], axis=1)

    # Number of Balls Faced
    total_balls_faced = delivery_df[["season", "match_number", "batter", "runs"]].groupby(by=["season", "match_number", "batter"], sort=False, as_index=False).count()
    total_balls_faced = total_balls_faced.rename(columns={"runs":"balls_faced"})
    wides_faced = full_extra_df[["season", "match_number", "batter", "wides"]].loc[np.where(full_extra_df["wides"] > 0)]
    noballs_faced = full_extra_df[["season", "match_number", "batter", "noballs"]].loc[np.where(full_extra_df["noballs"] > 0)]
    total_wides_faced = wides_faced.groupby(by=["season", "match_number", "batter"], sort=False, as_index=False).count()
    total_noballs_faced = noballs_faced.groupby(by=["season", "match_number", "batter"], sort=False, as_index=False).count()
    total_balls_faced = total_balls_faced.merge(total_wides_faced, how="left", on=["season", "match_number", "batter"])
    total_balls_faced = total_balls_faced.merge(total_noballs_faced, how="left", on=["season", "match_number", "batter"])
    total_balls_faced = total_balls_faced.fillna(0)
    total_balls_faced["balls_faced"] -= total_balls_faced["wides"] + total_balls_faced["noballs"]
    total_balls_faced = total_balls_faced.drop(["wides", "noballs"], axis=1)
    player_match_df = player_match_df.merge(total_balls_faced, 
                                            how="left", left_on=["season", "match_number", "player_id"], 
                                            right_on=["season", "match_number", "batter"])
    player_match_df = player_match_df.drop(["batter"], axis=1)

    # Number of Balls, No Balls, Wides Delivered
    total_balls_delivered = delivery_df[["season", "match_number", "bowler", "runs"]].groupby(by=["season", "match_number", "bowler"], sort=False, as_index=False).count()
    total_balls_delivered = total_balls_delivered.rename(columns={"runs":"balls_delivered"})
    wides_delivered = full_extra_df[["season", "match_number", "bowler", "wides"]].loc[np.where(full_extra_df["wides"] > 0)]
    noballs_delivered = full_extra_df[["season", "match_number", "bowler", "noballs"]].loc[np.where(full_extra_df["noballs"] > 0)]
    total_wides_delivered = wides_delivered.groupby(by=["season", "match_number", "bowler"], sort=False, as_index=False).count()
    total_noballs_delivered = noballs_delivered.groupby(by=["season", "match_number", "bowler"], sort=False, as_index=False).count()
    total_balls_delivered = total_balls_delivered.merge(total_wides_delivered, how="left", on=["season", "match_number", "bowler"])
    total_balls_delivered = total_balls_delivered.merge(total_noballs_delivered, how="left", on=["season", "match_number", "bowler"])
    total_balls_delivered = total_balls_delivered.fillna(0)
    total_balls_delivered["balls_delivered"] -= total_balls_delivered["wides"] + total_balls_delivered["noballs"]
    total_balls_delivered = total_balls_delivered.drop(["wides", "noballs"], axis=1)
    player_match_df = player_match_df.merge(total_balls_delivered, 
                                            how="left", left_on=["season", "match_number", "player_id"], 
                                            right_on=["season", "match_number", "bowler"])
    player_match_df = player_match_df.drop(["bowler"], axis=1)

    # Number of Wickets
    full_wickets_df = delivery_df.merge(wicket_df, on=["season", "match_number", "team_batting", "over", "number"])
    bowler_wickets = full_wickets_df.loc[np.where(full_wickets_df["type"] != "run out")]
    wickets_taken = bowler_wickets[["season", "match_number", "bowler", "wickets"]].groupby(by=["season", "match_number", "bowler"], sort=False, as_index=False).count()
    player_match_df = player_match_df.merge(wickets_taken, 
                                            how="left", left_on=["season", "match_number", "player_id"], 
                                            right_on=["season", "match_number", "bowler"])
    player_match_df = player_match_df.drop(["bowler"], axis=1)

    # Batsman Not-Out
    batsman_out = full_wickets_df[["season", "match_number", "player_out", "wickets"]].groupby(by=["season", "match_number", "player_out"], sort=False, as_index=False).count()
    batsman_out = batsman_out.rename(columns={"wickets":"out"})
    batsman_out["out"] = True
    player_match_df = player_match_df.merge(batsman_out, 
                                            how="left", left_on=["season", "match_number", "player_id"], 
                                            right_on=["season", "match_number", "player_out"])
    player_match_df = player_match_df.drop(["player_out"], axis=1)
    player_match_df["out"] = player_match_df["out"].fillna(False)
    player_match_df = player_match_df.fillna(0)

    # Batting Position
    delivery_df_chopped = delivery_df[["season", "match_number", "team_batting", "over", "number", "batter"]]
    # need to account for non-striker b/c they could be run out first ball + other edge cases
    delivery_df_chopped_ns = delivery_df[["season", "match_number", "team_batting", "over", "number", "non_striker"]]
    delivery_df_chopped_ns = delivery_df_chopped_ns.rename(columns={"non_striker":"batter"})
    delivery_df_chopped = pd.concat([delivery_df_chopped, delivery_df_chopped_ns], axis=0)
    player_position = delivery_df_chopped.sort_index()[["season", "match_number", "team_batting", "batter"]].drop_duplicates()
    player_position["position"] = player_position.groupby(by=["season", "match_number", "team_batting"]).cumcount()
    player_position["position"] += 1 # not zero-indexed as a standard, unlike delivery
    player_position = player_position.drop(["team_batting"], axis=1)
    player_match_df = player_match_df.merge(player_position, 
                                            how="left", left_on=["season", "match_number", "player_id"], 
                                            right_on=["season", "match_number", "batter"])
    player_match_df = player_match_df.drop(["batter"], axis=1)
    return player_match_df
pm_df = player_scorecards(pm_df)
pm_df

,name,player_id,season,match_number,team,runs_scored,runs_conceded,wides,noballs,fours_scored,fours_conceded,sixes_scored,sixes_conceded,balls_faced,balls_delivered,wickets,out,position
0,AK Markram,6a26221c,2024,23,Sunrisers Hyderabad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,True,3.0
1,Abdul Samad,8e514b4c,2024,23,Sunrisers Hyderabad,25.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,12.0,0.0,0.0,True,7.0
2,Abhishek Sharma,f29185a1,2024,23,Sunrisers Hyderabad,16.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,11.0,0.0,0.0,True,2.0
3,Arshdeep Singh,244048f6,2024,23,Punjab Kings,0.0,29.0,2.0,0.0,0.0,3.0,0.0,1.0,0.0,24.0,4.0,False,NaN
4,Ashutosh Sharma,84d9c311,2024,23,Punjab Kings,33.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,15.0,0.0,0.0,False,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24362,Shakib Al Hasan,7dc35884,2018,36,Sunrisers Hyderabad,0.0,34.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,24.0,0.0,False,NaN
24363,TA Boult,a818c1be,2018,36,Delhi Daredevils,0.0,30.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,24.0,0.0,False,NaN
24364,V Shankar,0994d0ae,2018,36,Delhi Daredevils,23.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,13.0,0.0,0.0,False,6.0
24365,WP Saha,fe11caa6,2018,36,Sunrisers Hyderabad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,NaN
